In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
#convert issue_d to numeric
import calendar

# For string columns
df['issue_mon'] = [x[0:3] for x in df['issue_d']]
df['next_pymnt_mon'] = [x[0:3] for x in df['next_pymnt_d']]

df = df.drop(['issue_d'], axis=1)
df = df.drop(['next_pymnt_d'], axis=1)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_mon,next_pymnt_mon
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,Mar,May
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,Mar,May
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,Mar,May
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,Mar,May
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,Mar,May


In [7]:
name_to_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}
name_to_num
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_mon,next_pymnt_mon
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,Mar,May
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,Mar,May
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,Mar,May
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,Mar,May
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,Mar,May


In [8]:
df["issue_mon"] = df["issue_mon"].apply(lambda x: name_to_num[x])
df["next_pymnt_mon"] = df["next_pymnt_mon"].apply(lambda x: name_to_num[x])
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_mon,next_pymnt_mon
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,3,5
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,3,5
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,3,5
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,3,5
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,3,5


In [9]:
# Binary encoding using Pandas (multiple column)
loans_binary_encoded = pd.get_dummies(df, columns=["home_ownership", "verification_status"])
loans_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,debt_settlement_flag,issue_mon,next_pymnt_mon,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,10500.0,0.1719,375.35,66000.0,low_risk,n,27.24,0.0,0.0,8.0,...,N,3,5,0,0,0,1,0,1,0
1,25000.0,0.2000,929.09,105000.0,low_risk,n,20.23,0.0,0.0,17.0,...,N,3,5,0,1,0,0,0,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,n,24.26,0.0,0.0,8.0,...,N,3,5,0,1,0,0,0,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,n,31.44,0.0,1.0,10.0,...,N,3,5,0,0,0,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,n,18.76,0.0,1.0,14.0,...,N,3,5,0,1,0,0,1,0,0


In [10]:
# Create the LabelEncoder instance and encode Education
#Low-risk = 1, high-risk = 0; pymnt_plan 1 = n, 0 = y; debt_settlement_flag and hardship_flag: 0 = N, 1 = Y
#application type is Individual (0) or Joint (1) & Initial List Status is Whole (1) or Fraction (0)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = loans_binary_encoded.copy()
df2['loan_status'] = le.fit_transform(loans_binary_encoded['loan_status'])
df2['initial_list_status'] = le.fit_transform(loans_binary_encoded['initial_list_status'])
df2['application_type'] = le.fit_transform(loans_binary_encoded['application_type'])
df2['pymnt_plan'] = le.fit_transform(loans_binary_encoded['pymnt_plan'])
df2['hardship_flag'] = le.fit_transform(loans_binary_encoded['hardship_flag'])
df2['debt_settlement_flag'] = le.fit_transform(loans_binary_encoded['debt_settlement_flag'])
df2.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,debt_settlement_flag,issue_mon,next_pymnt_mon,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,10500.0,0.1719,375.35,66000.0,1,0,27.24,0.0,0.0,8.0,...,0,3,5,0,0,0,1,0,1,0
1,25000.0,0.2000,929.09,105000.0,1,0,20.23,0.0,0.0,17.0,...,0,3,5,0,1,0,0,0,0,1
2,20000.0,0.2000,529.88,56000.0,1,0,24.26,0.0,0.0,8.0,...,0,3,5,0,1,0,0,0,0,1
3,10000.0,0.1640,353.55,92000.0,1,0,31.44,0.0,1.0,10.0,...,0,3,5,0,0,0,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,1,0,18.76,0.0,1.0,14.0,...,0,3,5,0,1,0,0,1,0,0


# Split the Data into Training and Testing

In [11]:
# Create our features
X = df2.drop(columns="loan_status")

# Create our target
y = df2["loan_status"]

In [12]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,debt_settlement_flag,issue_mon,next_pymnt_mon,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.0,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.0,1.726172,4.616839,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001
std,10277.348590,0.048130,288.062432,1.155800e+05,0.0,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.0,0.743862,0.486161,0.095914,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104
min,1000.000000,0.060000,30.890000,4.000000e+01,0.0,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.0,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.0,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.0,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,0.0,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.0,2.000000,5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,0.0,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.0,2.000000,5.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,0.0,999.000000,18.000000,5.000000,72.000000,4.000000,...,0.0,3.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
# Check the balance of our target values
#Low-risk = 1, high-risk = 0
df2['loan_status'].value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [16]:
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [17]:
y_pred = brf.predict(X_test)

In [18]:
# Calculated the balanced accuracy score (The balanced accuracy in binary and multiclass classification problems 
#to deal with imbalanced datasets. It is defined as the average of recall obtained on each class.)
balanced_accuracy_score(y_test, y_pred)

0.7531915989774842

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   64,    37],
       [ 2177, 14927]], dtype=int64)

In [20]:
# Print the imbalanced classification report (1 = low_risk, 0 = high_risk)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.63      0.87      0.05      0.74      0.54       101
          1       1.00      0.87      0.63      0.93      0.74      0.57     17104

avg / total       0.99      0.87      0.64      0.93      0.74      0.57     17205



In [21]:
# List the features sorted in descending order by feature importance
print(brf.feature_importances_)

[0.01148528 0.0295741  0.01716239 0.01544316 0.         0.02078134
 0.00392041 0.0041011  0.00817771 0.00133148 0.01570651 0.01163222
 0.00195169 0.0128821  0.01598144 0.06020967 0.06547195 0.06979047
 0.05232569 0.0077463  0.         0.         0.06775653 0.00042745
 0.         0.0022609  0.         0.00370442 0.01205494 0.00582389
 0.00666624 0.00718369 0.00749599 0.01503674 0.01269421 0.01534384
 0.00368077 0.0054775  0.01698648 0.01432051 0.01329543 0.00581731
 0.00623725 0.01198262 0.00744345 0.01344839 0.01415689 0.01358832
 0.         0.00010114 0.01808319 0.0146792  0.01053283 0.00895933
 0.00620953 0.01243825 0.01718232 0.00344348 0.00673987 0.00856112
 0.00675126 0.00894951 0.00902721 0.00970708 0.01013459 0.00853386
 0.00741312 0.         0.         0.00071642 0.00912229 0.0074882
 0.00636915 0.00156301 0.         0.01335735 0.01499366 0.01400374
 0.01222925 0.         0.         0.02577693 0.00512503 0.00014811
 0.00135723 0.00129779 0.00183574 0.00278414 0.00375683 0.00206

In [22]:
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.06979046868949672, 'total_rec_prncp'),
 (0.0677565266779018, 'last_pymnt_amnt'),
 (0.06547194787471669, 'total_pymnt_inv'),
 (0.06020967264258201, 'total_pymnt'),
 (0.052325691774331606, 'total_rec_int'),
 (0.029574103828736122, 'int_rate'),
 (0.025776928772848682, 'issue_mon'),
 (0.02078134072061495, 'dti'),
 (0.018083190114677454, 'mo_sin_old_il_acct'),
 (0.017182323711736572, 'mths_since_recent_inq'),
 (0.017162387110133747, 'installment'),
 (0.016986482987054564, 'max_bal_bc'),
 (0.015981441382303067, 'out_prncp_inv'),
 (0.015706505513655374, 'revol_bal'),
 (0.015443164031771277, 'annual_inc'),
 (0.015343839470907405, 'il_util'),
 (0.015036735113125777, 'mths_since_rcnt_il'),
 (0.01499365881779123, 'total_bal_ex_mort'),
 (0.014679200689452762, 'mo_sin_old_rev_tl_op'),
 (0.014320505178328079, 'all_util'),
 (0.014156892906438654, 'bc_open_to_buy'),
 (0.014003739914104618, 'total_bc_limit'),
 (0.013588318230404201, 'bc_util'),
 (0.01344839336431554, 'avg_cur_bal'),
 (0.01335735015

### Easy Ensemble AdaBoost Classifier

In [23]:
# Resample the training data, Train the EasyEnsembleClassifier, using 100 estimators
# This algorithm is known as EasyEnsemble[1]. The classifier is an ensemble of AdaBoost learners trained 
#on different balanced bootstrip samples.  the balancing is achieved by random under-sampling.

from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [25]:
# Calculated the balanced accuracy score (mean accuracy - In multi-label classification, this is the subset accuracy 
#which is a harsh metric since you require for each sample that each label set be correctly predicted. )

predictions = eec.predict(X_test)
balanced_accuracy_score(y_test, predictions)


0.9323908946086377

In [26]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[   93,     8],
       [  958, 16146]], dtype=int64)

In [28]:
# Print the imbalanced classification report (1 = low_risk, 0 = high_risk)
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.09      0.92      0.94      0.16      0.93      0.87       101
          1       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

